In [28]:
import torch
import torch.optim as optim
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import trange, tqdm_notebook
import copy
from torch.distributions.uniform import Uniform
from torch.distributions.normal import Normal
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.linalg import sqrtm
from torchvision import models
from torchvision import transforms
from PIL import Image

In [29]:
class I(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
    self.model.eval()

    #self.model = timm.create_model('inception_v3', pretrained=True, features_only=True)

  def forward(self, x):
    #https://discuss.pytorch.org/t/extract-the-2048-vector-of-a-fine-tuned-inception-v3-on-test-set/152510/4 
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    self.model.avgpool.register_forward_hook(get_activation("avgpool"))

    out = self.model(x)

    return activation['avgpool'].squeeze(3).squeeze(2)


In [57]:
class FID(nn.Module):
  def __init__(self):
    super().__init__()
    self.inception = I()
  
  def forward(self, x, y):
    with torch.no_grad():
      features_x = self.inception(x).detach().cpu().numpy()
      features_y = self.inception(y).detach().cpu().numpy()
      #print(features_x)
      #print(features_y)
      mu_x = features_x.mean(axis=0)
      mu_y = features_y.mean(axis=0)
      mu_part = np.dot(mu_x - mu_y, mu_x - mu_y)
      #print(mu_part)
      covariance_x = np.cov(features_x, rowvar=False)
      covariance_y = np.cov(features_y, rowvar=False)
      #print(covariance_x)
      #print(covariance_y)
      #print(np.multiply(covariance_x, covariance_y).shape)
      trace = covariance_x + covariance_y - 2 * sqrtm(np.dot(covariance_x, covariance_y))
      trace = np.trace(trace.real)
      #print(trace)
      fid = mu_part - trace
      #print(f"real fid: {fid}")
      #return features_x, features_y
      return fid


In [39]:
def fid_experiment():
  toTensorTransform = transforms.ToTensor()
  sample_number_1 = 2 #TODO correct this
  sample_number_2 = 2 #TODO correct this

  inputs1 = torch.zeros([sample_number_1, 3, 256, 256]) #TODO fix size
  for i in range(1, sample_number_1 + 1):
    img = Image.open("generated_sketch_" + str(i) + ".jpg")
    tensor = toTensorTransform(img)
    tensor = tensor.unsqueeze(0)
    inputs1[i] = tensor
  inputs1 = inputs1.cuda()

  inputs2 = torch.zeros([sample_number_2, 3, 256, 256]) #TODO fix size
  for i in range(1, sample_number_2 + 1):
    img = Image.open("true_sketch_" + str(i) + ".jpg")
    tensor = toTensorTransform(img)
    tensor = tensor.unsqueeze(0)
    inputs2[i] = tensor
  inputs2 = inputs2.cuda()

  fid = FID().cuda()
  fid_val = fid(inputs1, inputs2)

  return fid_val


In [58]:
fid_experiment()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


0.0002300919490065837

In [1]:
!pip install face_recognition
#https://github.com/ageitgey/face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 23 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=2c4cced1b4f2bf265ba3309954f2c713d69904110e7637685086e5524e1bf8e2
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [59]:
import face_recognition

def face_recognition_experiment():
  #I assume that known_sample_i corresponds to test_sample_i
  known_faces = []
  #TODO: how many samples exist? assign to sample_number
  sample_number = 3
  for i in range(1, sample_number + 1):
    image = face_recognition.load_image_file("known_sample_" + str(i) + ".jpg")
    face_encoding = face_recognition.face_encodings(image)[0]
    known_faces.append(face_encoding)

  number_of_accurates = 0
  for i in range(1, sample_number + 1):
    unknown_image = face_recognition.load_image_file("test_sample_" + str(i) + ".jpg")
    unknown_face_encoding = face_recognition.face_encodings(unknown_image)[0]
    results = face_recognition.compare_faces(known_faces, unknown_face_encoding)
    
    accurate = True
    for j, result in enumerate(results):
      accurate = accurate and ((j == i) == result)
      accurate = accurate and ((j != i) != result)
    #print(accurate)
    if accurate:
      number_of_accurates += 1 

  return number_of_accurates / sample_number


In [22]:
face_recognition_experiment()

True
True
False


0.6666666666666666